In [69]:
# Import your predictor
from linear_slope_predictor import LinearSlopePredictor
from window_peak_average import Average_window_max_predictor
# Example: from PREDICTOR_FILE import PREDICTOR_CLASS

# Add your predictor to the dictionary
predictors = {
    "LinearSlopePredictor": LinearSlopePredictor(),
    "window_peak_val": Average_window_max_predictor(),
    # Example: "PREDICTOR_CLASS": PREDICTOR_CLASS_CONSTRUCTOR_CALL
}

In [70]:
# Get raw data
import pandas as pd
from window_producer import raw_to_labeled_windows

raw_data = None
# If this fails, run get_raw_data.py 
raw_data = pd.read_csv("../combined_data.csv")
print(f"Raw datapoint count: {len(raw_data)}")

Raw datapoint count: 101754


In [71]:
# Filter raw data

# raw_data = raw_data[raw_data['class'] != 0]
raw_data.loc[raw_data['class'] != 2] = 0
raw_data.loc[raw_data['class'] == 2] = 1
raw_data = raw_data.rename(columns={
    "class": "y",
    "channel4": "x"
})
print(f"Filtered datapoint count: {len(raw_data)}")

Filtered datapoint count: 101754


In [72]:
# Label subset of raw data for train/test
train_data = raw_data.head(5000)
x_windows_train, y_labels_train = raw_to_labeled_windows(train_data, min_window_size=0, padding=0)

test_data = raw_data.tail(95000)
x_labels_test, y_labels_test = raw_to_labeled_windows(test_data, min_window_size=0, padding=0)

In [73]:
print(f"Train dataset size: {len(x_windows_train)}")
print(y_labels_train)

print(f"Test dataset size: {len(x_labels_test)}")
print(y_labels_test)

Train dataset size: 2
[0, 1]
Test dataset size: 7
[0, 1, 0, 1, 0, 1, 0]


In [74]:
# Train all predictors
for p in predictors:
    predictors[p].train(x_windows_train, y_labels_train)

In [75]:
# Test all predictors
predictor_results = {}
for p in predictors:
    predictor_results[p] = {
        'tp': 0,
        'fp': 0,
        'tn': 0,
        'fn': 0,
    }
    for i in range(len(x_labels_test)):
        pred = predictors[p].label_window(x_labels_test[i])
        actual = y_labels_test[i]

        if pred and actual:
            predictor_results[p]['tp'] += 1
        elif pred and not actual:
            predictor_results[p]['fp'] += 1
        elif not pred and not actual:
            predictor_results[p]['tn'] += 1
        elif not pred and actual:
            predictor_results[p]['fn'] += 1
print(predictor_results)

{'LinearSlopePredictor': {'tp': 3, 'fp': 4, 'tn': 0, 'fn': 0}, 'window_peak_val': {'tp': 3, 'fp': 0, 'tn': 4, 'fn': 0}}
